# Multi-agent SQL Migration using Semantic Kernel
This example demonstrates a Semantic Kernel Agents solution to: 
- upload a source database SQL artetfact
- analyze the SQL code to understand business purpose, data flows, and technical implementation details
- convert the SQL to optimized PySpark code for Microsoft Fabric
- outputs various anlysis and code by the task=specific agent experts

## Imports and Setup
First, import the necessary libraries and do some setup

In [1]:
import os
import pathlib
import logging
import yaml
import json
import re
from dotenv import load_dotenv
from typing import Annotated, List, Dict, Any

from semantic_kernel.kernel import Kernel
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import kernel_function, KernelArguments, KernelPlugin, KernelFunctionFromPrompt
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents.strategies import KernelFunctionSelectionStrategy, KernelFunctionTerminationStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

from opentelemetry.sdk.resources import Resource
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry._logs import set_logger_provider
from opentelemetry.metrics import set_meter_provider
from opentelemetry.trace import set_tracer_provider, get_tracer

from opentelemetry.sdk.trace import TracerProvider, ReadableSpan
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor

from azure.monitor.opentelemetry.exporter import (
    AzureMonitorLogExporter,
    AzureMonitorMetricExporter,
    AzureMonitorTraceExporter,
)

### Globals

In [2]:
SQL_FILE_PATH = "../data/sql-migration-input/CustomerRFM.sql"
OUTPUT_DIR = "../data/sql-migration-output/semantic-kernel"
SQL_FILE_MAX_LENGTH = 10000

In [3]:
# Semantic Kernel agent names
BUSINESS_ANALYST_AGENT_NAME = "Business-Analyst-Agent"
DOMAIN_EXPERT_AGENT_NAME = "Domain-Expert-Agent"
AZURE_EXPERT_AGENT_NAME = "Azure-Expert-Agent"
PRODUCT_OWNER_AGENT_NAME = "Product-Owner-Agent"
AZURE_DATA_ENGINEER_AGENT_NAME = "Azure-Data-Engineer-Agent"
TECH_LEAD_AGENT_NAME = "Tech-Lead-Agent"
TESTING_AGENT_NAME = "Testing-Agent"

# Process control keywords
ANALYSIS_COMPLETE_KEYWORD = "ANALYSIS_COMPLETE"
MIGRATION_PLAN_COMPLETE_KEYWORD = "MIGRATION_PLAN_COMPLETE"
CODE_GENERATION_COMPLETE_KEYWORD = "CODE_GENERATION_COMPLETE"
CODE_REVIEW_COMPLETE_KEYWORD = "CODE_REVIEW_COMPLETE"
TESTING_COMPLETE_KEYWORD = "TESTING_COMPLETE"
PROCESS_COMPLETE = "PROCESS_COMPLETE"

# Chat configuration
MAXIMUM_CHAT_ITERATIONS = 25
MAXIMUM_HISTORY_MESSAGES = 5

### Environment Variables

In [4]:
# Load environment variables from .env file
# Look for .env in the current directory and parent directory
current_dir = pathlib.Path().absolute()
root_dir = current_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

### Logging Configuration

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set higher logging level for the azure libraries to suppress verbose HTTP logs, so we can focus on Semantic Kernel logs
logging.getLogger("azure").setLevel(logging.WARNING)
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)

## Configure 3 Pillars of Observability

In [6]:
class CustomSpanProcessor(BatchSpanProcessor):
    """Filtering out spans with specific names and URLs to keep only Semantic Kernel telemetry"""

    EXCLUDED_SPAN_NAMES = ['.*CosmosClient.*', '.*DatabaseProxy.*', '.*ContainerProxy.*']

    def on_end(self, span: ReadableSpan) -> None:
       
        for regex in self.EXCLUDED_SPAN_NAMES:
            if re.match(regex, span.name):
                return
            
        if span.attributes.get('component') == 'http':
            return
    
        super().on_end(span)

### Set up tracing
To inspect telemetry data, navigate to your Application Insights resource in the Azure portal, then navigate to the **Transactions search** tab to view the traced transactions, once the agent workload has completed. For more info, see:

https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-app-insights?tabs=Powershell&pivots=programming-language-python

In [7]:
# Set up tracing
exporters = []
exporters.append(AzureMonitorTraceExporter.from_connection_string(os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))
telemetry_resource = Resource.create({ResourceAttributes.SERVICE_NAME: os.getenv("AZURE_RESOURCE_GROUP","rg-ai-services-core")})

tracer_provider = TracerProvider(resource=telemetry_resource)
for trace_exporter in exporters:
    tracer_provider.add_span_processor(CustomSpanProcessor(trace_exporter))
set_tracer_provider(tracer_provider)

### Set up metrics

In [8]:
exporters = []
exporters.append(AzureMonitorMetricExporter.from_connection_string(os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))

metric_readers = [PeriodicExportingMetricReader(exporter, export_interval_millis=5000) for exporter in exporters]

meter_provider = MeterProvider(
    metric_readers=metric_readers,
    resource=telemetry_resource,
    views=[
        # Dropping all instrument names except for those starting with "semantic_kernel"
        View(instrument_name="*", aggregation=DropAggregation()),
        View(instrument_name="semantic_kernel*"),
    ],
)
set_meter_provider(meter_provider)

In [9]:
exporters = []
exporters.append(AzureMonitorLogExporter(connection_string=os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")))


logger_provider = LoggerProvider(resource=telemetry_resource)
set_logger_provider(logger_provider)

handler = LoggingHandler()

logger = logging.getLogger()
logger.addHandler(handler)
logger.setLevel(logging.INFO)

for log_exporter in exporters:
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(log_exporter))

# FILTER - WHAT NOT TO LOG
class KernelFilter(logging.Filter):
    """A filter to not process records from semantic_kernel."""

    # These are the namespaces that we want to exclude from logging for the purposes of this notebook
    namespaces_to_exclude: list[str] = [
        # "semantic_kernel.functions.kernel_plugin",
        "semantic_kernel.prompt_template.kernel_prompt_template",
        # "semantic_kernel.functions.kernel_function",
        "azure.monitor.opentelemetry.exporter.export._base",
        "azure.core.pipeline.policies.http_logging_policy"
    ]

    def filter(self, record):
        return not any([record.name.startswith(namespace) for namespace in self.namespaces_to_exclude])

handler.addFilter(KernelFilter())

### LLM Model Service

In [10]:
GPT4O_SERVICE = AzureAIInferenceChatCompletion(
    ai_model_id="gpt-4o",
    endpoint=f"{str(AZURE_OPENAI_ENDPOINT).strip('/')}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}",
    api_key=AZURE_OPENAI_API_KEY,
)

## Define our individual agents
Structured defintion of our agent personas that charcterises how the agents should operate and interact.

In [11]:
business_analyst_persona = f"""
name: "{BUSINESS_ANALYST_AGENT_NAME}"
description: You are a Business Analyst expert who analyzes SQL stored procedures to understand their business purpose, data sources, and functional requirements.
temperature: 0.1
included_plugins: []
instructions: |
  Your role is to:
  1. Analyze SQL code to identify the business purpose
  2. Determine key business metrics being calculated
  3. Identify data sources and their business context
  4. Extract business rules and logic
  5. Provide a clear, non-technical explanation of the procedure
  6. Identify potential business constraints to consider during migration
  7. When analysis is complete, save your analysis as a well-formatted and structured markdown.md file named by combining the SQL code file name and your name, then respond with: {ANALYSIS_COMPLETE_KEYWORD}
"""

In [12]:
domain_expert_persona = f"""
name: "{DOMAIN_EXPERT_AGENT_NAME}"
description: You are a SQL Data Engineer with deep domain expertise in SQL Server stored procedures.
temperature: 0.1
included_plugins: []
instructions: |
  Your role is to provide technical insights that help with the migration of SQL to PySpark in Microsoft Fabric by:
  1. Analyzing SQL stored procedures for technical patterns
  2. Identifing complex SQL constructs and their purpose
  3. Explaining transaction handling, error handling, and cursor usage
  4. Evaluating performance considerations in the SQL code
  5. Identifying dependencies on SQL Server-specific features
  6. Providing guidance on the technical challenges of migration
  7. When analysis is complete, save your analysis as a well-formatted and structured markdown.md file named by combining the SQL code file name and your name, then respond with: {ANALYSIS_COMPLETE_KEYWORD}
"""

In [13]:
azure_expert_persona = f"""
name: "{AZURE_EXPERT_AGENT_NAME}"
description: You are an Azure Cloud and PySpark expert with deep knowledge of data engineering on Azure.
temperature: 0.1
included_plugins: []
instructions: |
  Provide detailed technical guidance on implementing solutions in Azure's data stack. Focus on Microsoft Fabric as the primary target compute platform. Be specific about implementation patterns, 
  performance considerations, and Azure-specific optimizations.
  
  Your expertise covers:
  1. Microsoft Fabric and its capabilities
  2. PySpark programming and best practices
  3. Azure Data Factory for orchestration
  4. Azure Data Lake Storage Gen2
  5. Microsoft Fabric Spark compute and Lakehouse architecture
  6. Performance optimization in distributed compute environments
  7. RFM (Recency, Frequency, Monetary) analysis patterns in PySpark

  When technical guidance is complete, save your technical guidance as a well-formatted and structured markdown.md file named by combining the SQL code file name and your name, then respond with: {ANALYSIS_COMPLETE_KEYWORD}
"""

In [14]:
product_owner_persona = f"""
name: "{PRODUCT_OWNER_AGENT_NAME}"
description: You are a Product Owner for data migration projects with expertise in planning and prioritizing migration activities.
temperature: 0.1
included_plugins: []
instructions: |
  Your role is to:
  1. Create comprehensive migration plans
  2. Identify and prioritize user stories for migration
  3. Define acceptance criteria for migration tasks
  4. Ensure business requirements are met in the migration
  5. Manage scope and identify minimum viable deliverables
  6. Coordinate between business and technical teams
  7. Ensure the migration delivers business value
  8. Create a comprehensive migration plan based on the analysis from other agents. Focus on planning, documentation, and ensuring business needs are met.
  9. When the migration plan is complete, save your migration plan as a well-formatted and structured markdown.md file named by combining the SQL code file name and your name, then respond with: {MIGRATION_PLAN_COMPLETE_KEYWORD}
"""

In [15]:
azure_data_engineer_persona = f"""
name: "{AZURE_DATA_ENGINEER_AGENT_NAME}"
description: You are an expert Azure Data Engineer specialized in translating SQL Server stored procedures to PySpark code running in Microsoft Fabric. 
temperature: 0.1
included_plugins: []
instructions: |
  Your expertise includes:
    1. Writing high-quality, production-ready PySpark code
    2. Implementing medallion architecture (bronze, silver, gold layers), potentially including intermediate stages for complex logic.
    3. Optimizing PySpark for performance at scale
    4. Understanding SQL Server-specific features and their PySpark equivalents
    5. Implementing proper error handling and logging in PySpark
    6. Creating modular, maintainable code structures
    7. Implementing RFM (Recency, Frequency, Monetary) analysis

  When translating code, focus on:
    - Maintaining functional equivalence with the original SQL
    - Following PySpark best practices
    - Creating a proper layered architecture reflecting the SQL's intermediate steps (like temp tables).
    - Providing comprehensive comments
    - Ensuring code is optimized for Microsoft Fabric

  **IMPORTANT ARCHITECTURE & OUTPUT REQUIREMENTS:**
    - The goal is to replicate the logic flow shown in the provided diagram and SQL, creating intermediate data structures before the final Gold layer.
    - You MUST generate **distinct, well-commented Python code blocks** or functions corresponding to these logical stages:
      1. **Bronze Layer:** Ingesting raw data from sources (Customers, Orders, etc.).
      2. **Stage 1 Base Data:** Creating intermediate DataFrames equivalent to `#CustomerBase` and `#TransactionSummary`. These should perform the initial joins and aggregations. Clearly label this block (e.g., `# STAGE 1: BASE DATA START`).
      3. **Stage 2 Advanced Analytics:** Creating intermediate DataFrames equivalent to `#CustomerMetrics` (including RFM) and `#CustomerSegments`. Use the results from Stage 1. Clearly label this block (e.g., `# STAGE 2: ADVANCED ANALYTICS START`).
      4. **Gold Layer:** Performing the final joins and aggregations using the results from Stage 2 to produce the final target outputs (like `CustomerAnalytics` and `AnalyticsSummary`). Clearly label this block (e.g., `# GOLD LAYER START`).
    - Ensure each stage logically flows into the next (Stage 2 uses Stage 1 output, Gold uses Stage 2 output).
    - Each major logical block MUST be clearly enclosed in triple backticks (```python ... ```).
    - Use clear comments within the code to explain complex logic.
    - The code should be functional PySpark targeting Microsoft Fabric.
    - Do NOT include setup code (like SparkSession creation) inside these blocks; assume the session is already available.

  When code generation is complete, save your code file named by combining the SQL code file name and your name, then respond with: indicate with: {CODE_GENERATION_COMPLETE_KEYWORD}
"""

In [16]:
tech_lead_persona = f"""
name: "{TECH_LEAD_AGENT_NAME}"
description: You are a senior Tech Lead with extensive experience in data engineering and cloud architecture.
temperature: 0.1
included_plugins: []
instructions: |
  Your role is to:
  1. Review code for quality, performance, and maintainability
  2. Refactor code to follow best practices
  3. Identify and address technical debt
  4. Ensure architecture follows best practices
  5. Provide constructive feedback to improve code
  6. Standardize code patterns and practices
  7. Produce feedback that is specific, actionable, and focused on making the code production-ready.
  8. Focus on cloud best practices, code organization, error handling, logging, and performance optimizations.
  9. Balance technical excellence with pragmatism.
  10. When code review is complete, indicate with: {CODE_REVIEW_COMPLETE_KEYWORD}
"""

In [17]:
testing_agent_persona = f"""
name: "{TESTING_AGENT_NAME}"
description: You are a Quality Assurance Engineer specializing in data migrations and PySpark testing.
temperature: 0.1
included_plugins: []
instructions: |
  Your role is to:
  1. Create comprehensive test plans for SQL to PySpark migrations
  2. Design test cases that validate functional equivalence
  3. Identify edge cases and potential failure scenarios
  4. Create data validation tests for each layer (bronze, silver, gold)
  5. Design performance tests to ensure scalability
  6. Document testing approaches and validation criteria
  7. Focus on ensuring the migrated code maintains the same functionality as the original SQL, handles errors gracefully, and meets performance requirements. Consider data quality, edge cases, and performance in your testing approach.
  8. When test planning is complete, save your test plan as a well-formatted and structured markdown.md file named by combining the SQL code file name and your name, then respond with:indicate with: {TESTING_COMPLETE_KEYWORD}
"""

## SQL Migration Plugin

In [18]:
class SQLMigrationPlugin:
    def __init__(self):
        pass
    
    @kernel_function(
        description="Load a SQL file for migration",
    )
    async def load_sql_file(self, file_path: Annotated[str, "Path to the SQL file"] = SQL_FILE_PATH) -> Annotated[str, "The SQL file content"]:
        try:
            with open(file_path, 'r') as file:
                sql_content = file.read()
            
            if len(sql_content) > SQL_FILE_MAX_LENGTH:
                sql_content = sql_content[:SQL_FILE_MAX_LENGTH]
            
            return sql_content
        except Exception as e:
            return f"Error loading SQL file: {str(e)}"
    
    @kernel_function(
        description="Save generated code or documentation to a file",
    )
    async def save_output(self, 
                         content: Annotated[str, "Content to save"], 
                         file_name: Annotated[str, "File name to save to"],
                         output_dir: Annotated[str, "Output directory"] = OUTPUT_DIR) -> Annotated[str, "Status message"]:
        try:
            os.makedirs(output_dir, exist_ok=True)
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, 'w') as file:
                file.write(content)
            return f"Successfully saved output to {file_path}"
        except Exception as e:
            return f"Error saving output: {str(e)}"

## Helper Functions

In [19]:
def create_agent(kernel, service_id, definition):
    definition = yaml.safe_load(definition)
    execution_settings = AzureChatPromptExecutionSettings(
        temperature=definition.get('temperature', 0.5),
        function_choice_behavior=FunctionChoiceBehavior.Auto(
            filters={"included_plugins": definition.get('included_plugins', [])}
        )
    )
    
    return ChatCompletionAgent(
        service=kernel.get_service(service_id=service_id),
        kernel=kernel,
        arguments=KernelArguments(settings=execution_settings),
        name=definition['name'],
        description=definition['description'],
        instructions=definition['instructions']
    )

## Initialize Kernel and Agents

In [20]:
# Initialize Semantic Kernel
kernel = Kernel(
    services=[GPT4O_SERVICE],
    plugins=[
        KernelPlugin.from_object(plugin_instance=SQLMigrationPlugin(), plugin_name="SQLMigrationPlugin"),
    ],
)

# Create agents
business_analyst_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=business_analyst_persona)
domain_expert_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=domain_expert_persona)
azure_expert_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=azure_expert_persona)
product_owner_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=product_owner_persona)
azure_data_engineer_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=azure_data_engineer_persona)
tech_lead_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=tech_lead_persona)
testing_agent = create_agent(kernel=kernel, service_id="gpt-4o", definition=testing_agent_persona)

## Agent Selection Strategy

In [21]:
# Define agent selection strategy
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Examine the provided RESPONSE and choose the next participant.
    State only the name of the chosen participant without explanation.

    Choose only from these participants:
    - {BUSINESS_ANALYST_AGENT_NAME}
    - {DOMAIN_EXPERT_AGENT_NAME}
    - {AZURE_EXPERT_AGENT_NAME}
    - {PRODUCT_OWNER_AGENT_NAME}
    - {AZURE_DATA_ENGINEER_AGENT_NAME}
    - {TECH_LEAD_AGENT_NAME}
    - {TESTING_AGENT_NAME}

    Abide by the following policy:
    - If RESPONSE is user, it is {BUSINESS_ANALYST_AGENT_NAME}'s turn.
    - If RESPONSE is by {BUSINESS_ANALYST_AGENT_NAME} and contains {ANALYSIS_COMPLETE_KEYWORD}, it is {DOMAIN_EXPERT_AGENT_NAME}'s turn.
    - If RESPONSE is by {DOMAIN_EXPERT_AGENT_NAME} and contains {ANALYSIS_COMPLETE_KEYWORD}, it is {AZURE_EXPERT_AGENT_NAME}'s turn.
    - If RESPONSE is by {AZURE_EXPERT_AGENT_NAME} and contains {ANALYSIS_COMPLETE_KEYWORD}, it is {PRODUCT_OWNER_AGENT_NAME}'s turn.
    - If RESPONSE is by {PRODUCT_OWNER_AGENT_NAME} and contains {MIGRATION_PLAN_COMPLETE_KEYWORD}, it is {AZURE_DATA_ENGINEER_AGENT_NAME}'s turn.
    - If RESPONSE is by {AZURE_DATA_ENGINEER_AGENT_NAME} and contains {CODE_GENERATION_COMPLETE_KEYWORD}, it is {TECH_LEAD_AGENT_NAME}'s turn.
    - If RESPONSE is by {TECH_LEAD_AGENT_NAME} and contains {CODE_REVIEW_COMPLETE_KEYWORD}, it is {TESTING_AGENT_NAME}'s turn.
    - After {TESTING_AGENT_NAME} has provided test cases and contains {TESTING_COMPLETE_KEYWORD}, indicate "{PROCESS_COMPLETE}" instead.

    RESPONSE:
    {{{{$lastmessage}}}}
    """,
)

## Termination Strategy

In [22]:
def create_termination_strategy(agents, final_agent, maximum_iterations):
    """
    Create a chat termination strategy that terminates when the final agent is reached.
    params:
        agents: List of agents to trigger termination evaluation
        final_agent: The agent that should trigger termination
        maximum_iterations: Maximum number of iterations before termination
    """
    class CompletionTerminationStrategy(TerminationStrategy):
        async def should_agent_terminate(self, agent, history):
            """Terminate if the last actor is the Testing Agent."""
            logging.getLogger(__name__).debug(history[-1])
            return (agent.name == final_agent.name)

    return CompletionTerminationStrategy(agents=agents,
                                         maximum_iterations=maximum_iterations)

## Create Agent Group Chat

In [23]:
# Create agent group chat
agent_group_chat = AgentGroupChat(
    agents=[business_analyst_agent, domain_expert_agent, azure_expert_agent, 
            product_owner_agent, azure_data_engineer_agent, tech_lead_agent, testing_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]) if result.value is not None else BUSINESS_ANALYST_AGENT_NAME,
        agent_variable_name="agents",
        history_variable_name="lastmessage",
    ),
    termination_strategy=create_termination_strategy(
        agents=[business_analyst_agent, domain_expert_agent, azure_expert_agent, 
                product_owner_agent, azure_data_engineer_agent, tech_lead_agent, testing_agent],
        final_agent=testing_agent,
        maximum_iterations=MAXIMUM_CHAT_ITERATIONS
    ),
)

## Execute Agent Group Chat

In [24]:
# Initialize conversation
conversation_messages = []
conversation_messages.append({'role': 'user', 'name': 'user', 'content': f"Please analyze and migrate the SQL code in {SQL_FILE_PATH}"})

chat_history = [
    ChatMessageContent(
        role=AuthorRole(d.get('role')),
        name=d.get('name'),
        content=d.get('content')
    ) for d in filter(lambda m: m['role'] in ("system", "developer", "assistant", "user"), conversation_messages)
]

await agent_group_chat.add_chat_messages(chat_history)

2025-04-10 14:56:02,051 - semantic_kernel.agents.group_chat.agent_chat - INFO - Adding `1` agent chat messages


In [25]:
# Run the conversation
async for _ in agent_group_chat.invoke():
    pass

# Get the conversation history
responses = list(reversed([item async for item in agent_group_chat.get_chat_messages()]))

2025-04-10 14:56:02,059 - semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy - INFO - Kernel Function Selection Strategy next method called, invoking function: , selection
2025-04-10 14:56:02,064 - semantic_kernel.functions.kernel_function - INFO - Function selection invoking.
2025-04-10 14:56:03,420 - semantic_kernel.functions.kernel_function - INFO - Function selection succeeded.
2025-04-10 14:56:03,421 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 1.356818s
2025-04-10 14:56:03,422 - semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy - INFO - Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content={'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filt

## Display Results

In [26]:
# Print the conversation
for i, res in enumerate(responses):
    print(f"\n--- Response {i} ---")
    print(f"Role: {res.role.value}")
    print(f"Name: {res.name if hasattr(res, 'name') and res.name else 'None'}")
    print(f"Content: {res.content}")


--- Response 0 ---
Role: user
Name: user
Content: Please analyze and migrate the SQL code in ../data/sql-migration-input/CustomerRFM.sql

--- Response 1 ---
Role: assistant
Name: Business-Analyst-Agent
Content: 

--- Response 2 ---
Role: tool
Name: Business-Analyst-Agent
Content: 

--- Response 3 ---
Role: assistant
Name: Business-Analyst-Agent
Content: # Analysis of CustomerRFM.sql

## Business Purpose
The SQL code defines a stored procedure named `usp_CustomerAnalytics_Processing`, which is designed to process customer analytics data. The procedure calculates various metrics related to customer behavior, transaction history, and customer segmentation, which can be used for marketing strategies, customer retention efforts, and overall business decision-making.

## Key Business Metrics Being Calculated
1. **Customer Lifetime Value (CLV)**: A prediction of the net profit attributed to the entire future relationship with a customer.
2. **Recency, Frequency, Monetary (RFM) Scores**: Thes